In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cuda')

In [4]:
from transformers import AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset, load_from_disk
import os
from transformers import TextDataset, DataCollatorForLanguageModeling



# Load pre-trained GPT-2 model and tokenizer
# model_name = "gpt2"
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


# To process dataset in one step, use Datasets map method to apply a preprocessing function over the entire dataset

if os.path.exists('./tokenized_datasets'):
    print("loading from disk : ./tokenized_datasets")
    tokenized_datasets = load_from_disk('./tokenized_datasets')
else:
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_datasets.save_to_disk("./tokenized_datasets")


In [ ]:
from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"   

do_ocr = True
if do_ocr:
    llmsherpa_api_url = llmsherpa_api_url + "&applyOcr=yes"

pdf_reader = LayoutPDFReader(llmsherpa_api_url)


In [ ]:

doc_obj = pdf_reader.read_pdf(file_path)

In [ ]:


# Process PDF data from books and user manuals
pdf_data_path = "path/to/pdf/files"
text_data = ""

for filename in os.listdir(pdf_data_path):
    if filename.endswith(".pdf"):
        with open(os.path.join(pdf_data_path, filename), "rb") as file:
            pdf_text = extract_text_from_pdf(file)
            text_data += pdf_text

# Tokenize the text data
tokenized_text = tokenizer(text_data, return_tensors="pt")

# Create a TextDataset from the tokenized text
dataset = TextDataset(tokenized_text, tokenizer=tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Define Trainer for unsupervised fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer),
    train_dataset=dataset,
)

# Perform unsupervised fine-tuning
trainer.train()

In [9]:
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

if os.path.exists('./google-bert_bert-base-cased'):
    model = AutoModelForSequenceClassification.from_pretrained('./google-bert_bert-base-cased')
else:
    model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased")
    model.save_pretrained('./google-bert_bert-base-cased')


In [23]:
modelQa = 'distilbert-base-cased-distilled-squad'

qa = pipeline('text-generation', model=model, tokenizer=tokenizer)

The model 'BertForSequenceClassification' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'Musicge

In [25]:
qa('which metrics are used to measure the performance of Picard + T5 model')

TypeError: The current model class (BertForSequenceClassification) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'BertLMHeadModel'}

In [26]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")

c:\Users\FUT_Novil\AppData\Local\anaconda3\envs\hfl\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FUT_Novil\.cache\huggingface\hub\models--google-bert--bert-large-uncased-whole-word-masking-finetuned-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at goo

In [28]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-base")

c:\Users\FUT_Novil\AppData\Local\anaconda3\envs\hfl\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FUT_Novil\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [35]:
# pipe('What are the metrics use to measuer the performace of T5 + Picard Model') #[{'generated_text': 'metric'}]
# pipe('What is RAG?') # [{'generated_text': 'racial agglomeration'}]
pipe('what are the differences between RAG and Fine tuning?') #[{'generated_text': 'RAG is a tuning technique that uses a computer to perform a precise tuning of'}]

c:\Users\FUT_Novil\AppData\Local\anaconda3\envs\hfl\lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'RAG is a tuning technique that uses a computer to perform a precise tuning of'}]